In [ ]:
import os
import glob
import trackpy as tp
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from pathlib import Path
from scipy.optimize import curve_fit
import numpy as np

framerate = 4

In [ ]:
file_directory = r"C:\Users\lizau\Desktop\walker_test\final_izbor\2b11\1a8\ROTATED\MSD" #insert the path to the directory with yur trajectory csv files
in_files = glob.glob(f'{file_directory}/*.csv')
out_dir = Path(f'{file_directory}\out_Y_vs_time')
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

trays_to_combine = []
max_particle_id = 0

for file in in_files:
    tray = pd.read_csv(file)
    if 'particle' in tray.columns:         # Offset the particle IDs to ensure uniqueness
        tray['particle'] += max_particle_id
        max_particle_id = tray['particle'].max() + 1
    trays_to_combine.append(tray)
combined_df = pd.concat(trays_to_combine, ignore_index=True)
combined_df = combined_df.sort_values(by='frame').reset_index(drop=True)
combined_df.to_csv(f'{out_dir}/combined_sorted_particles.csv')

print("DataFrames combined successfully!")

In [ ]:
#plots coordinate (user-defined) vs time

tray_len_query = 50
max_frame_plot = np.max(combined_df['frame'])
min_frame_plot = 0
coordinate = 'x'

particles_trays_filt_plot = combined_df.query(f'length>{tray_len_query} and {min_frame_plot}<frame<{max_frame_plot}')

# Get unique particle IDs with long trajectories
particles_to_plot = particles_trays_filt_plot['particle'].unique()

# Create the plot

plt.figure(figsize=(12, 6))
norm_coord_y = []
for pid in particles_to_plot:
    particle_data = particles_trays_filt_plot[particles_trays_filt_plot['particle'] == pid]
    if particle_data.empty:
        continue
    start_coord = particle_data[f'{coordinate}'].iloc[0]  # First y-coordinate
    normalized_coord = particle_data[f'{coordinate}'] - start_coord
    start_frame = particle_data['frame'].iloc[0]
    normalized_frame = particle_data['frame'] - start_frame
    if coordinate == 'x':
        start_y = particle_data['y'].iloc[0]
        normalized_y = particle_data['y'] - start_y
        norm_coord_y += [normalized_y]
    else:
        norm_coord_y += [normalized_coord]
    plt.plot(normalized_frame*0.25, normalized_coord*72, label=f'Particle {pid}')

# Customize plot
plt.xlabel('time [s]')
plt.ylabel(f'{coordinate} position [nm]')
plt.ylim(((np.min(np.concatenate(norm_coord_y))-0.2)*72, (np.max(np.concatenate(norm_coord_y))+0.2)*72))
plt.title(f'{coordinate} Position vs. Frame for Particles with Trajectory Length > {tray_len_query}')
plt.grid(True)
plt.tight_layout()
plt.savefig(f'{out_dir}/Normalized_{coordinate}_vs_time_frame_{min_frame_plot}_to_{max_frame_plot}_tray_len_{tray_len_query}.png')
plt.show()



In [ ]:
#plots histrogram of maximal trajectory offset for a user defined coordinate

tray_len_query = 10
max_frame_hist = np.max(combined_df['frame'])
min_frame_hist = 0
coordinate = 'x'

particles_trays_filt_hist = combined_df.query(f'length>{tray_len_query} and {min_frame_hist}<frame<{max_frame_hist}')
particles_to_plot = particles_trays_filt_hist['particle'].unique()

tray_offsets_joined = []
for pid in particles_to_plot:
    particle_data = particles_trays_filt_hist[particles_trays_filt_hist['particle'] == pid]
    tray_offset = (np.max(particle_data[f'{coordinate}']) - np.min(particle_data[f'{coordinate}']))*72
    tray_offsets_joined += [tray_offset]


plt.figure(figsize=(6, 6))
plt.hist(tray_offsets_joined, bins=10)
plt.xlabel('offset [nm]')
plt.title(f'{coordinate} max offset for trajectories longer than {tray_len_query} in frames {min_frame_hist} to {max_frame_hist}')
plt.savefig(f'{out_dir}/{coordinate}_max_offset_tray_len_{tray_len_query}_frame_{min_frame_hist}_to_{max_frame_hist}.png')
plt.show()



In [ ]:
#plots histrograms of maximal trajectory offset for for both coordinates on the same plot

tray_len_query = 10
max_frame_hist = np.max(combined_df['frame'])
min_frame_hist = 0

tray_offsets_x = []
tray_offsets_y = []

particles_trays_filt_hist = combined_df.query(f'length>{tray_len_query} and {min_frame_hist}<frame<{max_frame_hist}')
particles_to_plot = particles_trays_filt_hist['particle'].unique()

for pid in particles_to_plot:
    particle_data = particles_trays_filt_hist[particles_trays_filt_hist['particle'] == pid]
    
    if particle_data.empty:
        continue

    offset_x = (np.max(particle_data['x']) - np.min(particle_data['x'])) * 72
    offset_y = (np.max(particle_data['y']) - np.min(particle_data['y'])) * 72
    
    tray_offsets_x.append(offset_x)
    tray_offsets_y.append(offset_y)

# Plot histograms
plt.figure(figsize=(6, 6))
plt.hist(tray_offsets_x, bins=10, alpha=0.6, label='X offsets', color='blue')
plt.hist(tray_offsets_y, bins=10, alpha=0.6, label='Y offsets', color='orange')

plt.xlabel('Offset [nm]', fontsize=14)
plt.ylabel('Count', fontsize=14)
plt.title(f'Max X and Y Offsets for Trajectories > {tray_len_query} frames')
plt.legend()
plt.tick_params(axis='both', labelsize=14)
#plt.grid(True)
plt.tight_layout()

#plt.savefig(f'{out_dir}/xy_max_offset_tray_len_{tray_len_query}_frame_{min_frame_hist}_to_{max_frame_hist}.png')
plt.show()
